In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import itertools
from datetime import datetime
import seaborn as sns
from tqdm import tqdm
import math

pd.plotting.register_matplotlib_converters()
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Reshape, Dropout, BatchNormalization
#from keras.optimizers import Adam
from keras.optimizer_v2 import adam as adam_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
df = pd.read_csv("/kaggle/input/run-or-walk/dataset.csv")
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.isna().any()

In [ ]:
#check null data
null_data = df[df.isnull().any(axis=1)]
print(len(null_data))

In [ ]:
df['activity'].unique()

In [ ]:
#walk: 0
#run: 1
print("Walking:", (df['activity'] == 0).sum(axis=0))
print("Running:", (df['activity'] == 1).sum(axis=0))

In [ ]:
#combine date, time and drop unnescessary columns
#cols = ['date','time']
#df['combined'] = df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
df.drop(['date', 'time', 'username'], axis='columns', inplace=True)
df.head()

In [ ]:
#separate dataframe to two dataframes fro left wrist and right wrist
left_df = df[df.wrist == 0].copy()
left_df.drop(['wrist'], axis='columns', inplace = True)
right_df = df[df.wrist == 1].copy()
right_df.drop(['wrist'], axis='columns', inplace = True)
#print(left_df)
#print(right_df)

In [ ]:
# processing with right hand

Y = np.ravel(right_df['activity'])
X = right_df.iloc[:,1:]
print (type(Y))
print(type(X))
print (X.head())

In [ ]:
# scale the input
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
X = s.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test,Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=10) 

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from xgboost import XGBClassifier
gb = XGBClassifier(use_label_encoder=False)
gb.fit(X_train,Y_train)
Y_pred = gb.predict(X_test)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
gbacc = accuracy_score(Y_test,Y_pred)
print(gbacc)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
#Training using Gradient Boosting model using decision trees with the tf.estimator API

In [ ]:
#!pip3 install tensorflow_decision_forests --upgrade
#import tensorflow_decision_forests as tfdf
##train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train)
#print(type(X))
#train, test = train_test_split(right_df, test_size=0.25, random_state=10, shuffle=True)
#print(type(train))
#print (train.head())


In [ ]:
#tf_train = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="activity")
#tf_test = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="activity")


In [ ]:
#model = tfdf.keras.RandomForestModel()
# RandomForest is not implemented in Tensorflow Lite yet
#model.fit(tf_train)

# Summary of the model structure.
#model.summary()

# Evaluate the model.
#model.evaluate(tf_test)

In [ ]:
#model.save("project/rw_model")


In [ ]:
#convert the model
#converter = tf.lite.TFLiteConverter.from_saved_model("project/rw_model")
#tflite_model = converter.convert()

# Save the model.
#with open('rw_model.tflite', 'wb') as f:
#  f.write(tflite_model)

#saved_model_dir = ("project/rw_model")

In [ ]:
#converter = tf.lite.TFLiteConverter.from_saved_model(
    #saved_model_dir, signature_keys=['serving_default'])
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.experimental_new_converter = True
#converter.target_spec.supported_ops = [
    #tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#tflite_model = converter.convert()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(6, activation='relu', input_shape=(6,)))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
                   
model.fit(X_train, Y_train,epochs=4, batch_size=1, verbose=1)

In [ ]:
#from keras.utils import plot_model
#plot_model(model, to_file='/tmp/model.png', show_shapes=True,)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
score = model.evaluate(X_test, Y_test,verbose=1)

print(score)
print(model.summary())

for layer in model.layers:
    weights = layer.get_weights()
print(weights)

In [ ]:
saved_model_dir = ("project/rw_model")
tf.saved_model.save(model, saved_model_dir)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('rw_seq_model.tflite', 'wb') as f:
  f.write(tflite_model)